Họ và tên: Phạm Đức Thể

MSSV: 19522253

Lớp: DS307.N11

FINAL PROJECT - 18/11/2022

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1iMtJuBr4HbiwDmIkvr54sZKaULll_Gej?usp=sharing)

# **FINAL PROJECT - JOB RECOMMENDATION SYSTEM**

## **Import libraries**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import time

from datetime import date, datetime
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup

sns.set()

## **Read data**

In [2]:
PATH = '/content/drive/MyDrive/Dataset/DS307.N11/Final Project/'

In [ ]:
df_user = pd.read_csv(PATH+'Data_15000.csv', usecols=list(np.arange(2,19)), na_values='?')
df_user

,URL User,UserID,User Name,Industry,Desired Job,Workplace Desired,Current Place of Residence,Province,District,Desired Salary,Gender,Marriage,Date of Birth,Target,Skills,Degree,Work Experience
0,https://timviec365.vn/ung-vien/nguyen-quan-uv9...,964496,Nguyễn Quân,"[<span class=""ddmm"">Vận tải - Lái xe</span>, <...",Nhân viên lái xe bằng B2,['Hà Nội'],"Quận Ba Đình, Hà Nội",Hà Nội,Quận Ba Đình,Thỏa thuận,Nam,Đã có gia đình,03/01/1986,Với kinh nghiệm 10 năm trong lĩnh vực lái xe t...,"- Bằng lái xe hạng C, D, E và có kinh nghiệm 5...",Đơn vị đào tạo: Trường trung cấp nghề,Trên 5 năm
1,https://timviec365.vn/ung-vien/nguyen-thi-hong...,970215,Nguyễn Thị Hồng Nhung,"[<span class=""ddmm"">Sinh viên làm thêm</span>]",nhân viên tư vấn bán hàng,['Hà Nội'],"Nguyên Xá, Phường Minh Khai, QUận Bắc Từ Liêm,...",Hà Nội,Quận Bắc Từ Liêm,Thỏa thuận,NaN,NaN,NaN,NaN,NaN,NaN,Chưa có kinh nghiệm
2,https://timviec365.vn/ung-vien/le-hong-phu-uv9...,978880,Lê Hồng Phú,"[<span class=""ddmm"">Vận tải - Lái xe</span>]",Nhân viên lái xe bằng B2,['Hồ Chí Minh'],"Quận 10, hồ chí minh",Hồ Chí Minh,Quận 10,Thỏa thuận,Nam,Đã có gia đình,11/11/1994,NaN,NaN,NaN,1 năm
3,https://timviec365.vn/ung-vien/nguyen-chau-uv9...,986206,Nguyễn Châu,"[<span class=""ddmm"">Vận tải - Lái xe</span>, <...",Nhân viên lái xe bằng D,['Bình Dương'],"Thành phố Thuận An, Bình Dương",Bình Dương,Thành phố Thuận An,Thỏa thuận,Nam,Đã có gia đình,03/01/1978,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc\nCó khả năng...,Đơn vị đào tạo: Trường trung cấp nghề,Trên 10 năm
4,https://timviec365.vn/ung-vien/dang-tran-cong-...,985789,Đặng Trần Công,"[<span class=""ddmm"">Marketing - PR</span>, <sp...",HSE ENGINEER,['Khánh Hòa'],Hòa Phú,Phú Yên,Huyện Tây Hoà,Thỏa thuận,NaN,NaN,19/01/1996,NaN,NaN,NaN,1 năm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14863,https://timviec365.vn/ung-vien/nguyen-thi-doan...,985265,Nguyễn Thị Đoan Phương,"[<span class=""ddmm"">Công nghệ thực phẩm</span>]",Nhân viên kiểm soát chất lượng,"['Bình Dương', 'Tiền Giang', 'Bến Tre']",Bến Tre,Bến Tre,Huyện Giồng Trôm,Thỏa thuận,Nữ,Độc thân,21/12/1999,NaN,NaN,NaN,Dưới 1 năm
14864,https://timviec365.vn/ung-vien/vo-van-quang-uv...,961478,Võ Văn Quang,"[<span class=""ddmm"">IT phần mềm</span>]",Intern,['Hồ Chí Minh'],Ho Chi Minh City,Hồ Chí Minh,Quận Bình Tân,Thỏa thuận,NaN,NaN,NaN,NaN,NaN,NaN,Chưa có kinh nghiệm
14865,https://timviec365.vn/ung-vien/ho-minh-hoang-u...,982657,Hồ Minh Hoàng,"[<span class=""ddmm"">KD bất động sản</span>]",Phát triển dự án Hoặc phát triển và quản lý đạ...,['Quảng Trị'],"07 Trần Hoàn, P Đông Giang, TP Đông Hà, Quảng Trị",Quảng Trị,Thành phố Đông Hà,Thỏa thuận,NaN,NaN,NaN,NaN,NaN,NaN,Chưa có kinh nghiệm
14866,https://timviec365.vn/ung-vien/nguyen-thi-minh...,958279,Nguyễn Thị Minh Nguyệt,"[<span class=""ddmm"">Kế toán - Kiểm toán</span>]",kế toán nội bộ,['Hà Nội'],xuân mai,Hà Nội,Huyện Chương Mỹ,Thỏa thuận,NaN,NaN,NaN,NaN,NaN,NaN,Chưa có kinh nghiệm


In [ ]:
df_job = pd.read_csv(PATH+'job_data_full.csv')
df_job

,URL Job,Job Title,Name Company,Company Overview,Company Size,Company Address,Job Description,Job Requirements,Benefits,Job Address,Job Type,Gender,Number Cadidate,Career Level,Years of Experience,Salary,Submission Deadline,Industry
0,https://www.topcv.vn/viec-lam/sale-admin-websi...,Sale Admin Website,Công ty Cổ phần Belie,DHC Việt Nam trực thuộc Công ty CP Belie – Nhà...,100-499 nhân viên,"HN: Tầng 9, Tòa IDMC Mỹ Đình, 18 Tôn Thất Thuy...",Thường xuyên cập nhật các thay đổi liên quan t...,"1. Học vấn, kiến thức và trình độ chuyên môn:T...",Được xét duyệt đánh giá định kỳ 2 lần/năm.Làm ...,"['- Hà Nội: 18 Tôn Thất Thuyết, Nam Từ Liêm']",Toàn thời gian,Không yêu cầu,1 người,Nhân viên,1 năm,9-11 triệu,05/01/2023,"['Hành chính / Văn phòng', 'Kinh doanh / Bán h..."
1,https://www.topcv.vn/viec-lam/thuc-tap-sinh-la...,"Thực Tập Sinh Lập Trình (No-Code, Low-Code Pla...",Công Ty Cổ Phần Global Online Branding,Thành viên thuộc Tập đoàn G-Group:Công ty Cổ p...,100-499 nhân viên,"Lầu 6, 235 Lý Thường Kiệt ,Phường 6 ,Quận Tân ...","Phối hợp với bộ phận lập trình No-code, Low-co...","Sinh viên năm 3, năm 4 hoặc sinh viên cao học ...",Làm việc trong môi trường văn hóa Nhật Bản chu...,"['- Hồ Chí Minh: 235 Lý Thường Kiệt, Phường 6,...",Toàn thời gian,Không yêu cầu,2 người,Thực tập sinh,Không yêu cầu kinh nghiệm,Tới 1 triệu,01/01/2023,"['Công nghệ thông tin', 'IT phần mềm', 'Việc l..."
2,https://www.topcv.vn/viec-lam/hr-business-part...,HR Business Partner,Công Ty Melody Logistics,Công Ty TNHH Melody Logistics được biết đến là...,100-499 nhân viên,"Số 01 Nguyễn Văn Đậu Phường 5, Quận Phú Nhuận,...",Xây dựng và thực hiện chiến lược nhân sự phù h...,Tốt nghiệp đại học các chuyên ngành liên quan ...,Lương thỏa thuận theo năng lực;Làm việc trong ...,['- Hồ Chí Minh: Phú Nhuận'],Toàn thời gian,Không yêu cầu,1 người,Trưởng/Phó phòng,Trên 5 năm,Thoả thuận,28/12/2022,"['Nhân sự', 'Quản lý điều hành', 'Xuất nhập kh..."
3,https://www.topcv.vn/viec-lam/general-manager/...,General Manager,CÔNG TY TNHH DV HOLDINGS,"Khách sạn Danaciti By Dabi là khách sạn 4 sao,...",25-99 nhân viên,"80 Đông Kinh Nghĩa Thục, Sơn Trà, Đà Nẵng",1. Lập và triển khai kế hoạch kinh doanh- Định...,- Tốt nghiệp đại học trở lên chuyên ngành quản...,"- Tiền lương, thưởng và các khoản trợ cấp khác...","['- Đà Nẵng: 80 Đông Kinh Nghĩa Thục, Sơn Trà']",Toàn thời gian,Không yêu cầu,1 người,Giám đốc,3 năm,Thoả thuận,31/12/2022,"['Khách sạn / Nhà hàng', 'Thực phẩm / Đồ uống'..."
4,https://www.topcv.vn/viec-lam/le-tan-gymasaste...,Lễ Tân Gymasaster Quận 12,CÔNG TY TNHH TM XNK NGUỒN SỐNG VIỆT,Công ty TNHH TM XNK Nguồn Sống Việt cung câ...,100-499 nhân viên,"235 đường 9A, khu dân cư Trung Sơn, huyện Bi...",- Trực quầy Lễ tân- Tư vấn bán hàng gói member...,- Giới tính: Ưu tiên Nữ từ 22 - 35 tuổi- Kinh ...,Thu nhập = Lương cơ bản + Thưởng KPIs + Hoa hồ...,"['- Hồ Chí Minh: Gymaster 99 Phan Văn Hớn, Qu...",Toàn thời gian,Không yêu cầu,2 người,Nhân viên,Không yêu cầu kinh nghiệm,6-12 triệu,06/01/2023,"['Hành chính / Văn phòng', 'Dịch vụ khách hàng..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14629,https://timviec365.vn/viec-lam-ban-thoi-gian-t...,Việc Làm Bán Thời Gian Tại Nhà,công ty TNHH quảng cáo truyền thông xuyên việt,"Mô tả công ty Mô tả: chuyên về quảng cáo,...",Đang cập nhật,"88 Nguyễn Thái Học, Cầu Ông Lãnh, 1, Hồ Chí Minh",Mô tả công việc - Đối tượng: phù hợp với tất c...,Yêu cầu công việcBẠN SẼ PHÙ HỢP VỚI VỊ TRÍ NÀY...,Quyền lợi được hưởng Các Phúc Lợi Dành Cho Bạn...,Đắk Lắk,Bán thời gian,Không yêu cầu,8 ứng viên,Nhân viên,Không yêu cầu,7 - 10 triệu,29/12/2028,Sinh viên làm thêm
14630,https://timviec365.vn/nhan-vien-tu-van-tin-dun...,Nhân viên tư vấn Tín dụng qua điện thoại,Cty Cổ Phần Giải Pháp Nhân Sự Áng Sáng - Light...,Mô tả công ty,Đang cập nhật,"144 Tòa Nhà Fe Credit, Cộng Hòa, Tân Bình",Mô tả công việc 1.Mô tả công việc: - Sử dụng h...,Yêu cầu công việc- Tốt nghiệp THPT - Tuổi từ 1...,Quyền lợi được hưởng- Lương cơ bản: 5.100.000

## **Pre-processing data 1**

### **User data**

#### **Normalize data**

In [ ]:
def normalize_text_user(text):
  text = re.sub('\n+', ' ', text)
  text = re.sub('\xa0', ' ', text)
  text = re.sub('\r', '', text)
  text = re.sub('\t', '', text)
  text = re.sub('   ', '', text)
  text = re.sub('  ', ' ', text)
  text = text.strip()
  return text

| Chuẩn hóa mức lương mong muốn|
|-----|
|Thỏa thuận|
|1,000,000 - 5,000,000|
|5,000,000 - 10,000,000|
|10,000,000 - 15,000,000|
|15,000,000 - 20,000,000|
|20,000,000 - 40,000,000|
|Trên 40,000,000|

In [ ]:
def normalize_salary_user(salary):
  salary = salary.replace('.', '') 
  result = []
  if bool(re.search(r'\d', salary)): 
    if 'triệu' in salary:
      result.extend([int(j)*1000000 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    elif 'USD' in salary:
      result.extend([int(j)*23720 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    else:
      result.extend([int(j)*1000000 if int(j) < 100000 else int(j) for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
  else:
    result.append(salary)

  return result

In [ ]:
def calculate_age_user(born):
  today = date.today()
  return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

| Chuẩn hóa kinh nghiệm làm việc|
| ----- |
| Chưa có kinh nghiệm |
| Dưới 1 năm |
| 1-3 năm |
| 3-5 năm |
| 5-10 năm |
| Trên 10 năm |




In [ ]:
def normalize_experience_user(experience):
  try:
    result = 'Chưa có kinh nghiệm'
    number = int(re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', experience)[0])
    if bool(re.search(r'\d', experience)): 
      if 'Dưới' in experience:
        result = experience
      elif number < 3:
        result = '1-3 năm'
      elif number < 5:
        result = '3-5 năm'
      elif number < 10:
        result = '5-10 năm'
      else:
        result = 'Trên 10 năm'
  except:
    result = experience

  return result

In [ ]:
def processing_user_data(data):
  # Xóa cột Current Place of Residence, Province, District
  if 'Province' in data.columns:
    del data['Current Place of Residence']
    del data['Province']
    del data['District']
  # Xử lý missing values và dữ liệu trùng lặp
  data.dropna(subset=['Target', 'Skills'], inplace=True)
  data.drop_duplicates()
  data.reset_index(inplace=True)
  del data['index']
  # Xử lý dữ liệu trên từng cột
  for i in tqdm(range(data.shape[0])):
    try:
      # Xử lý cột User Name
      try:
        data.loc[i,'User Name'] = normalize_text_user(data.loc[i,'User Name'])
      except:
        data.loc[i,'User Name'] = 'Không xác định'
      # Xử lý cột Industry
      try:
        lst_industry = data.loc[i,'Industry'].split(',')
        if 'span' in lst_industry[0]:
          k = []
          for l in lst_industry:
            pattern = re.compile(r'\>(.+?)\<')
            k.extend(pattern.findall(l))
          data.loc[i,'Industry'] = ', '.join(k)
      except:
        data.loc[i,'Industry'] = 'Không xác định'
      # Xử lý cột Desired Job
      try:
        data.loc[i,'Desired Job'] = normalize_text_user(data.loc[i,'Desired Job'])
      except:
        data.loc[i,'Desired Job'] = 'Không xác định'
      # Xử lý cột Workplace Desired
      try:
        lst_workplace_desired = data.loc[i,'Workplace Desired'].split(',')
        if '[' in lst_workplace_desired[0]:
          k = []
          for l in lst_workplace_desired:
            pattern = re.compile(r"\'(.+?)\'")
            k.extend(pattern.findall(l))
          data.loc[i,'Workplace Desired'] = ', '.join(k)
      except:
        data.loc[i,'Workplace Desired'] = 'Không xác định'
      # Xử lý cột Desired Salary
      salary = min(normalize_salary_user(data.loc[i,'Desired Salary']))
      try:
        if salary < 5000000:
          data.loc[i,'Desired Salary'] = '1,000,000 - 5,000,000'
        elif salary < 10000000:
          data.loc[i,'Desired Salary'] = '5,000,000 - 10,000,000'
        elif salary < 15000000:
          data.loc[i,'Desired Salary'] = '10,000,000 - 15,000,000'
        elif salary < 20000000:
          data.loc[i,'Desired Salary'] = '15,000,000 - 20,000,000'
        elif salary < 40000000:
          data.loc[i,'Desired Salary'] = '20,000,000 - 40,000,000'
        else:
          data.loc[i,'Desired Salary'] = 'Trên 40,000,000'
      except:
        data.loc[i,'Desired Salary'] = salary
      # Xử lý cột Gender
      data['Gender'].fillna(value='Khác', inplace = True)
      # Xử lý cột Marriage
      data['Marriage'].fillna(value='Khác', inplace = True)
      # Xử lý cột Date of Birth -> chuyển thành cột Age
      if 'Date of Birth' in data.columns:
        try:
          DATE = datetime.strptime(data.loc[i,'Date of Birth'],"%d/%m/%Y").date()
          data.loc[i,'Date of Birth'] = calculate_age_user(DATE)
        except:
          data.loc[i,'Date of Birth'] = data.loc[i,'Date of Birth']
      # Xử lý cột Target
      data.loc[i,'Target'] = normalize_text_user(data.loc[i,'Target'])
      # Xử lý cột Skills
      data.loc[i,'Skills'] = normalize_text_user(data.loc[i,'Skills'])
      # Xử lý cột Target
      data.loc[i,'Degree'] = normalize_text_user(data.loc[i,'Degree'])
      # Xử lý cột Work Experience
      data.loc[i,'Work Experience'] = normalize_experience_user(data.loc[i,'Work Experience'])
    except:
      continue
  # Đổi tên cột Date of Birth -> Age
  data.rename(columns={'Date of Birth': 'Age'}, inplace=True)
  return data

In [ ]:
%%time
processing_user_data(df_user)

100%|██████████| 3983/3983 [00:24<00:00, 164.02it/s]

CPU times: user 24.3 s, sys: 952 ms, total: 25.3 s
Wall time: 24.3 s


,URL User,UserID,User Name,Industry,Desired Job,Workplace Desired,Desired Salary,Gender,Marriage,Age,Target,Skills,Degree,Work Experience
0,https://timviec365.vn/ung-vien/nguyen-quan-uv9...,964496,Nguyễn Quân,"Vận tải - Lái xe, Vận chuyển giao nhận",Nhân viên lái xe bằng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,36,Với kinh nghiệm 10 năm trong lĩnh vực lái xe t...,"- Bằng lái xe hạng C, D, E và có kinh nghiệm 5...",Đơn vị đào tạo: Trường trung cấp nghề,5-10 năm
1,https://timviec365.vn/ung-vien/nguyen-chau-uv9...,986206,Nguyễn Châu,"Vận tải - Lái xe, Vận chuyển giao nhận",Nhân viên lái xe bằng D,Bình Dương,Thỏa thuận,Nam,Đã có gia đình,44,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trường trung cấp nghề,Trên 10 năm
2,https://timviec365.vn/ung-vien/la-thi-hoang-uv...,769632,LÃ THỊ HOÀNG,"Hành chính - Văn phòng, Biên - Phiên dịch, Khu...","Nhân viên văn phòng, khu công nghiệp sử dụng t...",Bắc Giang,Thỏa thuận,Nữ,Độc thân,25,- Tìm kiếm một vị trí nhân viên hành chính và ...,- Am hiểu mọi quy trình về hành chính nhân sự ...,Mô tả công việc: CÔNG TY CỔ PHẦN DƯỢC PHẨM AN ...,1-3 năm
3,https://timviec365.vn/ung-vien/nguyen-van-bao-...,981314,Nguyễn Văn Bảo,Vận tải - Lái xe,Nhân viên lái xe hạng C,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,33,Trở thành kỹ sư tốt trong công tác sửa chữa xe...,"Có khả năng chu toàn công việc, siêng năng, tỉ...",Đơn vị đào tạo: Trung tâm dạy nghề,5-10 năm
4,https://timviec365.vn/ung-vien/nguyen-thi-tu-u...,962892,Nguyễn Thị Tứ,"Quản lý đơn hàng, Thống kê, Việc làm bán hàng",Nhân viên thu mua,Hồ Chí Minh,Thỏa thuận,Nữ,Độc thân,24,"- Có một công việc có nhiều cơ hội thăng tiến,...","- Trung thực, cẩn thận, có óc sáng tạo - Kỹ...",Đơn vị đào tạo: Cao Đẳng Kinh Tế Kỹ Thuật Vina...,3-5 năm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,https://timviec365.vn/ung-vien/bui-nghia-uv997...,997433,Bùi Nghĩa,"Vận tải - Lái xe, Vận chuyển giao nhận",Nhân viên lái xe hạng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,35,"Trở thành nhân viên lái xe cho công ty, tôi mu...","- Tiếp thu tốt các kiến thức lái xe, - Có trí ...",Đơn vị đào tạo: Trường trung cấp nghề,1-3 năm
3979,https://timviec365.vn/ung-vien/tran-thanh-son-...,964381,Trần Thanh Sơn,"Vận tải - Lái xe, Vận chuyển giao nhận",Nhân viên Lái xe hạng C,Khánh Hòa,Thỏa thuận,Nam,Đã có gia đình,29,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trung học,5-10 năm
3980,https://timviec365.vn/ung-vien/huynh-qui-uv969...,969486,Huỳnh Quí,"Vận tải - Lái xe, Vận chuyển giao nhận",Nhân viên lái xe bằng B2,Hồ Chí Minh,Thỏa thuận,Nam,Độc thân,20,Áp dụng những kiến thức về giao thông vận tải ...,"Khả năng chịu áp lực cao trong công việc, có t...",Đơn vị đào tạo: Trường trung cấp nghề,Dưới 1 năm
3981,https://timviec365.vn/ung-vien/pham-van-truong...,978744,Phạm Văn Trường,Vận tải - Lái xe,Nhân viên lái xe - Bằng Lái Xe Hạng D,Quảng Ninh,Thỏa thuận,Nam,Đã có gia đình,34,"Đáp ứng tốt các công việc của công ty, giữ gìn...","Giao tiếp tốt, thân thiện, cởi mở với mọi ngườ...",Đơn vị đào tạo: Tốt nghiệp THPT,Trên 10 năm


#### **Check & Processing Missing Values**

In [ ]:
df_user.isna().sum()

URL User              0
UserID                0
User Name             0
Industry              0
Desired Job           0
Workplace Desired     0
Desired Salary        0
Gender                0
Marriage              0
Age                  59
Target                0
Skills                0
Degree                0
Work Experience       0
dtype: int64

In [ ]:
df_user[df_user['Industry'] == '']

,URL User,UserID,User Name,Industry,Desired Job,Workplace Desired,Desired Salary,Gender,Marriage,Age,Target,Skills,Degree,Work Experience
14,https://timviec365.vn/ung-vien/nguyen-thanh-xu...,958757,Nguyễn Thanh Xuân,,Nhân viên Làm việc online,Hồ Chí Minh,Thỏa thuận,Nữ,Đã có gia đình,32,- Mong muốn được làm việc trong một môi trường...,"- Trung thực, cẩn thận, có óc sáng tạo - Kỹ...",Đơn vị đào tạo: Đại Học Nguyễn Tất Thành Chuyê...,Chưa có kinh nghiệm
806,https://timviec365.vn/ung-vien/tran-ngo-hoan-h...,960754,Trần Ngô Hoàn Hảo,,bán quần áo,"Hồ Chí Minh, Hồ Chí Minh","5,000,000 - 10,000,000",Nữ,Độc thân,19,Bán quần áoMong muốn tìm được chỗ làm ổn định ...,Nổ lực cần cù bù thông minhBiết lắng nghe tiếp...,PTCS Đơn vị đào tạo: Đăng khoa,1-3 năm
1589,https://timviec365.vn/ung-vien/tran-ngo-hoan-h...,960754,Trần Ngô Hoàn Hảo,,bán quần áo,"Hồ Chí Minh, Hồ Chí Minh","5,000,000 - 10,000,000",Nữ,Độc thân,19,Bán quần áoMong muốn tìm được chỗ làm ổn định ...,Nổ lực cần cù bù thông minhBiết lắng nghe tiếp...,PTCS Đơn vị đào tạo: Đăng khoa,1-3 năm
3322,https://timviec365.vn/ung-vien/vu-thi-thuong-u...,990783,Vu thi Thuong,,bán vé máy bay,Hà Nội,"5,000,000 - 10,000,000",Nữ,Đã có gia đình,32,Mong muốn tìm được chỗ làm ổn định lâu dài,Biết lắng nghe tiếp thu ý kiến của đồng nghiệp...,Đơn vị đào tạo: cao đẳngXếp loại tốt nghiệp:,Chưa có kinh nghiệm


In [ ]:
df_user['Age'].fillna(0, inplace = True)
df_user['Industry'].replace('', 'Không xác định', inplace = True)

In [ ]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3983 entries, 0 to 3982
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   URL User           3983 non-null   object
 1   UserID             3983 non-null   int64 
 2   User Name          3983 non-null   object
 3   Industry           3983 non-null   object
 4   Desired Job        3983 non-null   object
 5   Workplace Desired  3983 non-null   object
 6   Desired Salary     3983 non-null   object
 7   Gender             3983 non-null   object
 8   Marriage           3983 non-null   object
 9   Age                3983 non-null   int64 
 10  Target             3983 non-null   object
 11  Skills             3983 non-null   object
 12  Degree             3983 non-null   object
 13  Work Experience    3983 non-null   object
dtypes: int64(2), object(12)
memory usage: 435.8+ KB


#### **Save**

In [ ]:
# Save data
# df_user.to_csv(PATH+'USER_DATA_T.csv', index=False, encoding='utf-8')

### **Job data**

#### **Normalize data**

In [ ]:
def normalize_text_job(text):
  text = re.sub('\n+', ' ', text)
  text = re.sub('\xa0', ' ', text)
  text = re.sub('\r', '', text)
  text = re.sub('\t', '', text)
  text = re.sub('   ', '', text)
  text = re.sub('  ', ' ', text)
  text = text.strip()
  return text

| Chuẩn hóa quy mô công ty|
| ----- |
| 1-9 |
| 10-24 |
| 25-99 |
| 100-499 |
| 500-999 |
| 1000-3000 |
| 3000+ |

In [ ]:
def normalize_company_size(size):
  try:
    result = []
    number = int(re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', size)[0])
    if bool(re.search(r'\d', size)):
      if 'Ít' in size or number < 10:
        size = '1-9'
      elif number < 25:
        size = '10-24'
      elif number < 100:
        size = '25-99'
      elif number < 500:
        size = '100-499'
      elif number < 1000:
        size = '500-999'
      elif number < 3000:
        size = '1000-3000'
      else:
        size = '3000+'
  except:
    size = 'Đang cập nhật'
  return size

In [ ]:
def normalize_address(job_address):
  if '[' in job_address:
    k = []
    job_address = re.sub(" '',", '', job_address)
    job_address = re.sub("'Thu gọn'", '', job_address)
    pattern = re.compile(r"\'(.+?)\'")
    k.extend(pattern.findall(job_address))
    job_address = ', '.join(k)
  
  return job_address

| Chuẩn hóa loại hình làm việc|
| ----- |
| Part time |
| Full time |
| Remote |
| Others |




In [ ]:
def normalize_job_type(job_type):
  if 'Bán thời gian' in job_type:
    job_type = 'Part time'
  elif 'Toàn thời gian' in job_type:
    job_type = 'Full time'
  elif 'Việc làm từ xa' in job_type:
    job_type = 'Remote'
  else:
    job_type = 'Others'
  
  return job_type

| Chuẩn hóa mức lương|
|-----|
|Thỏa thuận|
|1,000,000 - 5,000,000|
|5,000,000 - 10,000,000|
|10,000,000 - 15,000,000|
|15,000,000 - 20,000,000|
|20,000,000 - 40,000,000|
|Trên 40,000,000|

In [ ]:
def normalize_salary_job(salary):
  salary = salary.replace('.', '') 
  salary = salary.replace(',', '')
  salary = salary.replace('Thoả thuận', 'Thỏa thuận')
  result = []
  if bool(re.search(r'\d', salary)): 
    if 'triệu' in salary:
      result.extend([int(j)*1000000 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    elif 'USD' in salary:
      result.extend([int(j)*23720 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    else:
      result.extend([int(j)*1000000 if int(j) < 100000 else int(j) for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
  else:
    result.append(salary)

  return result

| Chuẩn hóa kinh nghiệm làm việc|
| ----- |
| Không yêu cầu kinh nghiệm |
| Dưới 1 năm |
| 1-3 năm |
| 3-5 năm |
| 5-10 năm |
| Trên 10 năm |




In [ ]:
def normalize_experience_job(experience):
  try:
    result = 'Không yêu cầu kinh nghiệm'
    number = int(re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', experience)[0])
    if bool(re.search(r'\d', experience)): 
      if 'Dưới' in experience or number < 1:
        result = 'Dưới 1 năm'
      elif number < 3:
        result = '1-3 năm'
      elif number < 5:
        result = '3-5 năm'
      elif number < 10:
        result = '5-10 năm'
      else:
        result = 'Trên 10 năm'
  except:
    result = 'Không yêu cầu kinh nghiệm'

  return result

In [ ]:
def normalize_industry(industry):
  if '[' in industry:
    k = []
    pattern = re.compile(r"\'(.+?)\'")
    k.extend(pattern.findall(industry))
    industry = ', '.join(k)
  
  return industry

In [ ]:
def processing_job_data(data):
  # Xử lý dữ liệu trên từng cột
  for i in tqdm(range(data.shape[0])):
    try:
      # Xử lý cột Job Title
      data.loc[i,'Job Title'] = normalize_text_job(data.loc[i,'Job Title'])
      # Xử lý cột Name Company
      data.loc[i,'Name Company'] = normalize_text_job(data.loc[i,'Name Company'])
      # Xử lý cột Company Overview
      try:
        data.loc[i,'Company Overview'] = re.sub('Mô tả công ty', '', data.loc[i,'Company Overview'])
        try:
          data.loc[i,'Company Overview'] = normalize_text_job(data.loc[i,'Company Overview'])
        except:
          data.loc[i,'Company Overview'] = 'Đang cập nhật'
      except:
        data.loc[i,'Company Overview'] = 'Đang cập nhật'  
      # Xử lý cột Company Size
      data.loc[i,'Company Size'] = normalize_company_size(data.loc[i,'Company Size'])
      # Xử lý cột Company Address
      data.loc[i,'Company Address'] = normalize_text_job(data.loc[i,'Company Address'])
      # Xử lý cột Job Description
      data.loc[i,'Job Description'] = re.sub('Mô tả công việc', '', data.loc[i,'Job Description'])
      data.loc[i,'Job Description'] = normalize_text_job(data.loc[i,'Job Description'])
      # Xử lý cột Job Requirements
      data.loc[i,'Job Requirements'] = re.sub('Yêu cầu công việc', '', data.loc[i,'Job Requirements'])
      data.loc[i,'Job Requirements'] = normalize_text_job(data.loc[i,'Job Requirements'])
      # Xử lý cột Benefits
      data.loc[i,'Benefits'] = re.sub('Quyền lợi được hưởng', '', data.loc[i,'Benefits'])
      data.loc[i,'Benefits'] = normalize_text_job(data.loc[i,'Benefits'])
      # Xử lý cột Job Address
      data.loc[i,'Job Address'] = normalize_address(data.loc[i,'Job Address'])
      # Xử lý cột Job Type
      data.loc[i,'Job Type'] = normalize_job_type(data.loc[i,'Job Type'])
      # Xử lý cột Gender
      data.loc[i,'Gender'] = re.sub('0', 'Không yêu cầu', data.loc[i,'Gender'])
      # Xử lý cột Number Cadidate
      data.loc[i,'Number Cadidate'] = int(re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', data.loc[i,'Number Cadidate'])[0])
      # Xử lý cột Years of Experience
      data.loc[i,'Years of Experience'] = normalize_experience_job(data.loc[i,'Years of Experience'])
      # Xử lý cột Salary
      salary = min(normalize_salary_job(data.loc[i,'Salary']))
      try:
        if salary < 5000000:
          data.loc[i,'Salary'] = '1,000,000 - 5,000,000'
        elif salary < 10000000:
          data.loc[i,'Salary'] = '5,000,000 - 10,000,000'
        elif salary < 15000000:
          data.loc[i,'Salary'] = '10,000,000 - 15,000,000'
        elif salary < 20000000:
          data.loc[i,'Salary'] = '15,000,000 - 20,000,000'
        elif salary < 40000000:
          data.loc[i,'Salary'] = '20,000,000 - 40,000,000'
        else:
          data.loc[i,'Salary'] = 'Trên 40,000,000'
      except:
        data.loc[i,'Salary'] = salary
      # Xử lý cột Submission Deadline
      data.loc[i,'Submission Deadline'] = normalize_text_job(data.loc[i,'Submission Deadline'])
      # Xử lý cột Industry
      try:
        data.loc[i,'Industry'] = normalize_industry(data.loc[i,'Industry'])
      except:
        data.loc[i,'Industry'] = 'Không xác định'
    except:
      continue

  return data

In [ ]:
%%time
processing_job_data(df_job)

100%|██████████| 14634/14634 [01:01<00:00, 239.06it/s]

CPU times: user 59.1 s, sys: 2.25 s, total: 1min 1s
Wall time: 1min 1s


,URL Job,Job Title,Name Company,Company Overview,Company Size,Company Address,Job Description,Job Requirements,Benefits,Job Address,Job Type,Gender,Number Cadidate,Career Level,Years of Experience,Salary,Submission Deadline,Industry
0,https://www.topcv.vn/viec-lam/sale-admin-websi...,Sale Admin Website,Công ty Cổ phần Belie,DHC Việt Nam trực thuộc Công ty CP Belie – Nhà...,100-499,"HN: Tầng 9, Tòa IDMC Mỹ Đình, 18 Tôn Thất Thuy...",Thường xuyên cập nhật các thay đổi liên quan t...,"1. Học vấn, kiến thức và trình độ chuyên môn:T...",Được xét duyệt đánh giá định kỳ 2 lần/năm.Làm ...,"- Hà Nội: 18 Tôn Thất Thuyết, Nam Từ Liêm",Full time,Không yêu cầu,1,Nhân viên,1-3 năm,"5,000,000 - 10,000,000",05/01/2023,"Hành chính / Văn phòng, Kinh doanh / Bán hàng,..."
1,https://www.topcv.vn/viec-lam/thuc-tap-sinh-la...,"Thực Tập Sinh Lập Trình (No-Code, Low-Code Pla...",Công Ty Cổ Phần Global Online Branding,Thành viên thuộc Tập đoàn G-Group:Công ty Cổ p...,100-499,"Lầu 6, 235 Lý Thường Kiệt ,Phường 6 ,Quận Tân ...","Phối hợp với bộ phận lập trình No-code, Low-co...","Sinh viên năm 3, năm 4 hoặc sinh viên cao học ...",Làm việc trong môi trường văn hóa Nhật Bản chu...,"- Hồ Chí Minh: 235 Lý Thường Kiệt, Phường 6, Q...",Full time,Không yêu cầu,2,Thực tập sinh,Không yêu cầu kinh nghiệm,"1,000,000 - 5,000,000",01/01/2023,"Công nghệ thông tin, IT phần mềm, Việc làm IT"
2,https://www.topcv.vn/viec-lam/hr-business-part...,HR Business Partner,Công Ty Melody Logistics,Công Ty TNHH Melody Logistics được biết đến là...,100-499,"Số 01 Nguyễn Văn Đậu Phường 5, Quận Phú Nhuận,...",Xây dựng và thực hiện chiến lược nhân sự phù h...,Tốt nghiệp đại học các chuyên ngành liên quan ...,Lương thỏa thuận theo năng lực;Làm việc trong ...,- Hồ Chí Minh: Phú Nhuận,Full time,Không yêu cầu,1,Trưởng/Phó phòng,5-10 năm,Thỏa thuận,28/12/2022,"Nhân sự, Quản lý điều hành, Xuất nhập khẩu"
3,https://www.topcv.vn/viec-lam/general-manager/...,General Manager,CÔNG TY TNHH DV HOLDINGS,"Khách sạn Danaciti By Dabi là khách sạn 4 sao,...",25-99,"80 Đông Kinh Nghĩa Thục, Sơn Trà, Đà Nẵng",1. Lập và triển khai kế hoạch kinh doanh- Định...,- Tốt nghiệp đại học trở lên chuyên ngành quản...,"- Tiền lương, thưởng và các khoản trợ cấp khác...","- Đà Nẵng: 80 Đông Kinh Nghĩa Thục, Sơn Trà",Full time,Không yêu cầu,1,Giám đốc,3-5 năm,Thỏa thuận,31/12/2022,"Khách sạn / Nhà hàng, Thực phẩm / Đồ uống, Spa..."
4,https://www.topcv.vn/viec-lam/le-tan-gymasaste...,Lễ Tân Gymasaster Quận 12,CÔNG TY TNHH TM XNK NGUỒN SỐNG VIỆT,Công ty TNHH TM XNK Nguồn Sống Việt cung câ...,100-499,"235 đường 9A, khu dân cư Trung Sơn, huyện Bi...",- Trực quầy Lễ tân- Tư vấn bán hàng gói member...,- Giới tính: Ưu tiên Nữ từ 22 - 35 tuổi- Kinh ...,Thu nhập = Lương cơ bản + Thưởng KPIs + Hoa hồ...,"- Hồ Chí Minh: Gymaster 99 Phan Văn Hớn, Quận 12",Full time,Không yêu cầu,2,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",06/01/2023,"Hành chính / Văn phòng, Dịch vụ khách hàng, Tư..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14629,https://timviec365.vn/viec-lam-ban-thoi-gian-t...,Việc Làm Bán Thời Gian Tại Nhà,công ty TNHH quảng cáo truyền thông xuyên việt,"Mô tả: chuyên về quảng cáo,...",Đang cập nhật,"88 Nguyễn Thái Học, Cầu Ông Lãnh, 1, Hồ Chí Minh",- Đối tượng: phù hợp với tất cả mọi người như:...,BẠN SẼ PHÙ HỢP VỚI VỊ TRÍ NÀY NẾU BẠN: - Có kh...,Các Phúc Lợi Dành Cho Bạn : -Lương : Làm việc ...,Đắk Lắk,Part time,Không yêu cầu,8,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",29/12/2028,Sinh viên làm thêm
14630,https://timviec365.vn/nhan-vien-tu-van-tin-dun...,Nhân viên tư vấn Tín dụng qua điện thoại,Cty Cổ Phần Giải Pháp Nhân Sự Áng Sáng - Light...,,Đang cập nhật,"144 Tòa Nhà Fe Credit, Cộng Hòa, Tân Bình",1.: - Sử dụng hệ thống Call-center của công ty...,- Tốt nghiệp THPT - Tuổi từ 19-33t- Kỹ năng gi...,- Lương cơ bản: 5.100.000/Tháng - Phụ cấp cơm:...,Hồ Chí Minh,Full time,Không yêu cầu,100,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,0

#### **Check & Processing Missing Values**

In [ ]:
df_job.isna().sum()

URL Job                0
Job Title              0
Name Company           0
Company Overview       0
Company Size           0
Company Address        0
Job Description        0
Job Requirements       0
Benefits               0
Job Address            0
Job Type               0
Gender                 0
Number Cadidate        0
Career Level           0
Years of Experience    0
Salary                 0
Submission Deadline    0
Industry               0
dtype: int64

In [ ]:
df_job['Company Overview'].replace('', 'Đang cập nhật', inplace = True)
df_job['Job Requirements'].replace('', 'Đang cập nhật', inplace = True)
df_job['Benefits'].replace('', 'Đang cập nhật', inplace = True)
df_job['Job Address'].replace('', 'Đang cập nhật', inplace = True)

In [ ]:
df_job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14634 entries, 0 to 14633
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   URL Job              14634 non-null  object
 1   Job Title            14634 non-null  object
 2   Name Company         14634 non-null  object
 3   Company Overview     14634 non-null  object
 4   Company Size         14634 non-null  object
 5   Company Address      14634 non-null  object
 6   Job Description      14634 non-null  object
 7   Job Requirements     14634 non-null  object
 8   Benefits             14634 non-null  object
 9   Job Address          14634 non-null  object
 10  Job Type             14634 non-null  object
 11  Gender               14634 non-null  object
 12  Number Cadidate      14634 non-null  object
 13  Career Level         14634 non-null  object
 14  Years of Experience  14634 non-null  object
 15  Salary               14634 non-null  object
 16  Subm

#### **Save**

In [ ]:
# Save data
# df_job.to_csv(PATH+'JOB_DATA_T.csv', index=False, encoding='utf-8')

## **Pre-processing data 2**

### **User data**

#### **Normalize data**

In [ ]:
def normalize_text_user(text):
  text = re.sub('\n+', ' ', text)
  text = re.sub('\xa0', ' ', text)
  text = re.sub('\r', '', text)
  text = re.sub('\t', '', text)
  text = re.sub('   ', '', text)
  text = re.sub('  ', ' ', text)
  text = text.strip()
  return text

| Chuẩn hóa mức lương mong muốn|
|-----|
|Thỏa thuận|
|1,000,000 - 5,000,000|
|5,000,000 - 10,000,000|
|10,000,000 - 15,000,000|
|15,000,000 - 20,000,000|
|20,000,000 - 40,000,000|
|Trên 40,000,000|

In [ ]:
def normalize_salary_user(salary):
  salary = salary.replace('.', '') 
  result = []
  if bool(re.search(r'\d', salary)): 
    if 'triệu' in salary:
      result.extend([int(j)*1000000 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    elif 'USD' in salary:
      result.extend([int(j)*23720 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    else:
      result.extend([int(j)*1000000 if int(j) < 100000 else int(j) for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
  else:
    result.append(salary)

  return result

In [ ]:
def calculate_age_user(born):
  today = date.today()
  return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

| Chuẩn hóa kinh nghiệm làm việc|
| ----- |
| Chưa có kinh nghiệm |
| Dưới 1 năm |
| 1-3 năm |
| 3-5 năm |
| 5-10 năm |
| Trên 10 năm |




In [ ]:
def normalize_experience_user(experience):
  try:
    result = 'Chưa có kinh nghiệm'
    number = int(re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', experience)[0])
    if bool(re.search(r'\d', experience)): 
      if 'Dưới' in experience:
        result = experience
      elif number < 3:
        result = '1-3 năm'
      elif number < 5:
        result = '3-5 năm'
      elif number < 10:
        result = '5-10 năm'
      else:
        result = 'Trên 10 năm'
  except:
    result = experience

  return result

In [ ]:
def processing_user_data(data):
  # Xóa cột Current Place of Residence, Province, District
  if 'Province' in data.columns:
    del data['Current Place of Residence']
    del data['Province']
    del data['District']
  # Xử lý missing values và dữ liệu trùng lặp
  data.dropna(subset=['Target', 'Skills'], inplace=True)
  data.drop_duplicates()
  data.reset_index(inplace=True)
  del data['index']
  # Xử lý dữ liệu trên từng cột
  for i in tqdm(range(data.shape[0])):
    try:
      # Xử lý cột User Name
      try:
        data.loc[i,'User Name'] = normalize_text_user(data.loc[i,'User Name'])
      except:
        data.loc[i,'User Name'] = 'Không xác định'
      # Xử lý cột Industry
      try:
        lst_industry = data.loc[i,'Industry'].split(',')
        if 'span' in lst_industry[0]:
          k = []
          for l in lst_industry:
            pattern = re.compile(r'\>(.+?)\<')
            k.extend(pattern.findall(l))
          data.loc[i,'Industry'] = k[0]
      except:
        data.loc[i,'Industry'] = 'Không xác định'
      # Xử lý cột Desired Job
      try:
        data.loc[i,'Desired Job'] = normalize_text_user(data.loc[i,'Desired Job'])
      except:
        data.loc[i,'Desired Job'] = 'Không xác định'
      # Xử lý cột Workplace Desired
      try:
        lst_workplace_desired = data.loc[i,'Workplace Desired'].split(',')
        if '[' in lst_workplace_desired[0]:
          k = []
          for l in lst_workplace_desired:
            pattern = re.compile(r"\'(.+?)\'")
            k.extend(pattern.findall(l))
          data.loc[i,'Workplace Desired'] = ', '.join(k)
      except:
        data.loc[i,'Workplace Desired'] = 'Không xác định'
      # Xử lý cột Desired Salary
      salary = min(normalize_salary_user(data.loc[i,'Desired Salary']))
      try:
        if salary < 5000000:
          data.loc[i,'Desired Salary'] = '1,000,000 - 5,000,000'
        elif salary < 10000000:
          data.loc[i,'Desired Salary'] = '5,000,000 - 10,000,000'
        elif salary < 15000000:
          data.loc[i,'Desired Salary'] = '10,000,000 - 15,000,000'
        elif salary < 20000000:
          data.loc[i,'Desired Salary'] = '15,000,000 - 20,000,000'
        elif salary < 40000000:
          data.loc[i,'Desired Salary'] = '20,000,000 - 40,000,000'
        else:
          data.loc[i,'Desired Salary'] = 'Trên 40,000,000'
      except:
        data.loc[i,'Desired Salary'] = salary
      # Xử lý cột Gender
      data['Gender'].fillna(value='Khác', inplace = True)
      # Xử lý cột Marriage
      data['Marriage'].fillna(value='Khác', inplace = True)
      # Xử lý cột Date of Birth -> chuyển thành cột Age
      if 'Date of Birth' in data.columns:
        try:
          DATE = datetime.strptime(data.loc[i,'Date of Birth'],"%d/%m/%Y").date()
          data.loc[i,'Date of Birth'] = calculate_age_user(DATE)
        except:
          data.loc[i,'Date of Birth'] = data.loc[i,'Date of Birth']
      # Xử lý cột Target
      data.loc[i,'Target'] = normalize_text_user(data.loc[i,'Target'])
      # Xử lý cột Skills
      data.loc[i,'Skills'] = normalize_text_user(data.loc[i,'Skills'])
      # Xử lý cột Target
      data.loc[i,'Degree'] = normalize_text_user(data.loc[i,'Degree'])
      # Xử lý cột Work Experience
      data.loc[i,'Work Experience'] = normalize_experience_user(data.loc[i,'Work Experience'])
    except:
      continue
  # Đổi tên cột Date of Birth -> Age
  data.rename(columns={'Date of Birth': 'Age'}, inplace=True)
  return data

In [ ]:
%%time
processing_user_data(df_user)

100%|██████████| 3983/3983 [00:25<00:00, 156.96it/s]

CPU times: user 24.1 s, sys: 701 ms, total: 24.8 s
Wall time: 25.5 s


,URL User,UserID,User Name,Industry,Desired Job,Workplace Desired,Desired Salary,Gender,Marriage,Age,Target,Skills,Degree,Work Experience
0,https://timviec365.vn/ung-vien/nguyen-quan-uv9...,964496,Nguyễn Quân,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,36,Với kinh nghiệm 10 năm trong lĩnh vực lái xe t...,"- Bằng lái xe hạng C, D, E và có kinh nghiệm 5...",Đơn vị đào tạo: Trường trung cấp nghề,5-10 năm
1,https://timviec365.vn/ung-vien/nguyen-chau-uv9...,986206,Nguyễn Châu,Vận tải - Lái xe,Nhân viên lái xe bằng D,Bình Dương,Thỏa thuận,Nam,Đã có gia đình,44,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trường trung cấp nghề,Trên 10 năm
2,https://timviec365.vn/ung-vien/la-thi-hoang-uv...,769632,LÃ THỊ HOÀNG,Hành chính - Văn phòng,"Nhân viên văn phòng, khu công nghiệp sử dụng t...",Bắc Giang,Thỏa thuận,Nữ,Độc thân,25,- Tìm kiếm một vị trí nhân viên hành chính và ...,- Am hiểu mọi quy trình về hành chính nhân sự ...,Mô tả công việc: CÔNG TY CỔ PHẦN DƯỢC PHẨM AN ...,1-3 năm
3,https://timviec365.vn/ung-vien/nguyen-van-bao-...,981314,Nguyễn Văn Bảo,Vận tải - Lái xe,Nhân viên lái xe hạng C,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,33,Trở thành kỹ sư tốt trong công tác sửa chữa xe...,"Có khả năng chu toàn công việc, siêng năng, tỉ...",Đơn vị đào tạo: Trung tâm dạy nghề,5-10 năm
4,https://timviec365.vn/ung-vien/nguyen-thi-tu-u...,962892,Nguyễn Thị Tứ,Quản lý đơn hàng,Nhân viên thu mua,Hồ Chí Minh,Thỏa thuận,Nữ,Độc thân,24,"- Có một công việc có nhiều cơ hội thăng tiến,...","- Trung thực, cẩn thận, có óc sáng tạo - Kỹ...",Đơn vị đào tạo: Cao Đẳng Kinh Tế Kỹ Thuật Vina...,3-5 năm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,https://timviec365.vn/ung-vien/bui-nghia-uv997...,997433,Bùi Nghĩa,Vận tải - Lái xe,Nhân viên lái xe hạng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,35,"Trở thành nhân viên lái xe cho công ty, tôi mu...","- Tiếp thu tốt các kiến thức lái xe, - Có trí ...",Đơn vị đào tạo: Trường trung cấp nghề,1-3 năm
3979,https://timviec365.vn/ung-vien/tran-thanh-son-...,964381,Trần Thanh Sơn,Vận tải - Lái xe,Nhân viên Lái xe hạng C,Khánh Hòa,Thỏa thuận,Nam,Đã có gia đình,29,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trung học,5-10 năm
3980,https://timviec365.vn/ung-vien/huynh-qui-uv969...,969486,Huỳnh Quí,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hồ Chí Minh,Thỏa thuận,Nam,Độc thân,20,Áp dụng những kiến thức về giao thông vận tải ...,"Khả năng chịu áp lực cao trong công việc, có t...",Đơn vị đào tạo: Trường trung cấp nghề,Dưới 1 năm
3981,https://timviec365.vn/ung-vien/pham-van-truong...,978744,Phạm Văn Trường,Vận tải - Lái xe,Nhân viên lái xe - Bằng Lái Xe Hạng D,Quảng Ninh,Thỏa thuận,Nam,Đã có gia đình,34,"Đáp ứng tốt các công việc của công ty, giữ gìn...","Giao tiếp tốt, thân thiện, cởi mở với mọi ngườ...",Đơn vị đào tạo: Tốt nghiệp THPT,Trên 10 năm


#### **Check & Processing Missing Values**

In [ ]:
df_user.isna().sum()

URL User              0
UserID                0
User Name             0
Industry              0
Desired Job           0
Workplace Desired     0
Desired Salary        0
Gender                0
Marriage              0
Age                  59
Target                0
Skills                0
Degree                0
Work Experience       0
dtype: int64

In [ ]:
df_user['Age'].fillna(0, inplace = True)

In [ ]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3983 entries, 0 to 3982
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   URL User           3983 non-null   object
 1   UserID             3983 non-null   int64 
 2   User Name          3983 non-null   object
 3   Industry           3983 non-null   object
 4   Desired Job        3983 non-null   object
 5   Workplace Desired  3983 non-null   object
 6   Desired Salary     3983 non-null   object
 7   Gender             3983 non-null   object
 8   Marriage           3983 non-null   object
 9   Age                3983 non-null   int64 
 10  Target             3983 non-null   object
 11  Skills             3983 non-null   object
 12  Degree             3983 non-null   object
 13  Work Experience    3983 non-null   object
dtypes: int64(2), object(12)
memory usage: 435.8+ KB


#### **Save**

In [ ]:
# Save data
# df_user.to_csv(PATH+'USER_DATA_TN.csv', index=False, encoding='utf-8')

### **Job data**

#### **Normalize data**

In [ ]:
def normalize_text_job(text):
  text = re.sub('\n+', ' ', text)
  text = re.sub('\xa0', ' ', text)
  text = re.sub('\r', '', text)
  text = re.sub('\t', '', text)
  text = re.sub('   ', '', text)
  text = re.sub('  ', ' ', text)
  text = text.strip()
  return text

| Chuẩn hóa quy mô công ty|
| ----- |
| 1-9 |
| 10-24 |
| 25-99 |
| 100-499 |
| 500-999 |
| 1000-3000 |
| 3000+ |

In [ ]:
def normalize_company_size(size):
  try:
    result = []
    number = int(re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', size)[0])
    if bool(re.search(r'\d', size)):
      if 'Ít' in size or number < 10:
        size = '1-9'
      elif number < 25:
        size = '10-24'
      elif number < 100:
        size = '25-99'
      elif number < 500:
        size = '100-499'
      elif number < 1000:
        size = '500-999'
      elif number < 3000:
        size = '1000-3000'
      else:
        size = '3000+'
  except:
    size = 'Đang cập nhật'
  return size

In [ ]:
def normalize_address(job_address):
  k = []
  try:
    if '[' in job_address:
      job_address = re.sub(" '',", '', job_address)
      job_address = re.sub("'Thu gọn'", '', job_address)
      if len(job_address) > 20:
        pattern = re.compile(r"\-(.+?)\:")
        k.extend(pattern.findall(job_address))
        job_address = k[0].strip()
      else:
        pattern = re.compile(r"\-(.+?)\'")
        k.extend(pattern.findall(job_address))
        job_address = k[0].strip()
  except:
    pattern = re.compile(r"\-(.+?)\'")
    k.extend(pattern.findall(job_address))
    job_address = ''.join(k)
    job_address = job_address.strip()
  job_address = re.sub('\W+',' ', job_address)
  return normalize_text_job(job_address)

| Chuẩn hóa loại hình làm việc|
| ----- |
| Part time |
| Full time |
| Remote |
| Others |




In [ ]:
def normalize_job_type(job_type):
  if 'Bán thời gian' in job_type:
    job_type = 'Part time'
  elif 'Toàn thời gian' in job_type:
    job_type = 'Full time'
  elif 'Việc làm từ xa' in job_type:
    job_type = 'Remote'
  else:
    job_type = 'Others'
  
  return job_type

| Chuẩn hóa mức lương|
|-----|
|Thỏa thuận|
|1,000,000 - 5,000,000|
|5,000,000 - 10,000,000|
|10,000,000 - 15,000,000|
|15,000,000 - 20,000,000|
|20,000,000 - 40,000,000|
|Trên 40,000,000|

In [ ]:
def normalize_salary_job(salary):
  salary = salary.replace('.', '') 
  salary = salary.replace(',', '')
  salary = salary.replace('Thoả thuận', 'Thỏa thuận')
  result = []
  if bool(re.search(r'\d', salary)): 
    if 'triệu' in salary:
      result.extend([int(j)*1000000 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    elif 'USD' in salary:
      result.extend([int(j)*23720 for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
    else:
      result.extend([int(j)*1000000 if int(j) < 100000 else int(j) for j in re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', salary)])
  else:
    result.append(salary)

  return result

| Chuẩn hóa kinh nghiệm làm việc|
| ----- |
| Không yêu cầu kinh nghiệm |
| Dưới 1 năm |
| 1-3 năm |
| 3-5 năm |
| 5-10 năm |
| Trên 10 năm |




In [ ]:
def normalize_experience_job(experience):
  try:
    result = 'Không yêu cầu kinh nghiệm'
    number = int(re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', experience)[0])
    if bool(re.search(r'\d', experience)): 
      if 'Dưới' in experience or number < 1:
        result = 'Dưới 1 năm'
      elif number < 3:
        result = '1-3 năm'
      elif number < 5:
        result = '3-5 năm'
      elif number < 10:
        result = '5-10 năm'
      else:
        result = 'Trên 10 năm'
  except:
    result = 'Không yêu cầu kinh nghiệm'

  return result

In [ ]:
def normalize_industry(industry):
  if '[' in industry:
    k = []
    pattern = re.compile(r"\'(.+?)\'")
    k.extend(pattern.findall(industry))
    industry = k[0]
  
  return industry

In [ ]:
def processing_job_data(data):
  # Xử lý dữ liệu trên từng cột
  for i in tqdm(range(data.shape[0])):
    try:
      # Xử lý cột Job Title
      data.loc[i,'Job Title'] = normalize_text_job(data.loc[i,'Job Title'])
      # Xử lý cột Name Company
      data.loc[i,'Name Company'] = normalize_text_job(data.loc[i,'Name Company'])
      # Xử lý cột Company Overview
      try:
        data.loc[i,'Company Overview'] = re.sub('Mô tả công ty', '', data.loc[i,'Company Overview'])
        try:
          data.loc[i,'Company Overview'] = normalize_text_job(data.loc[i,'Company Overview'])
        except:
          data.loc[i,'Company Overview'] = 'Đang cập nhật'
      except:
        data.loc[i,'Company Overview'] = 'Đang cập nhật'  
      # Xử lý cột Company Size
      data.loc[i,'Company Size'] = normalize_company_size(data.loc[i,'Company Size'])
      # Xử lý cột Company Address
      data.loc[i,'Company Address'] = normalize_text_job(data.loc[i,'Company Address'])
      # Xử lý cột Job Description
      data.loc[i,'Job Description'] = re.sub('Mô tả công việc', '', data.loc[i,'Job Description'])
      data.loc[i,'Job Description'] = normalize_text_job(data.loc[i,'Job Description'])
      # Xử lý cột Job Requirements
      data.loc[i,'Job Requirements'] = re.sub('Yêu cầu công việc', '', data.loc[i,'Job Requirements'])
      data.loc[i,'Job Requirements'] = normalize_text_job(data.loc[i,'Job Requirements'])
      # Xử lý cột Benefits
      data.loc[i,'Benefits'] = re.sub('Quyền lợi được hưởng', '', data.loc[i,'Benefits'])
      data.loc[i,'Benefits'] = normalize_text_job(data.loc[i,'Benefits'])
      # Xử lý cột Job Address
      data.loc[i,'Job Address'] = normalize_address(data.loc[i,'Job Address'])
      # Xử lý cột Job Type
      data.loc[i,'Job Type'] = normalize_job_type(data.loc[i,'Job Type'])
      # Xử lý cột Gender
      data.loc[i,'Gender'] = re.sub('0', 'Không yêu cầu', data.loc[i,'Gender'])
      # Xử lý cột Number Cadidate
      data.loc[i,'Number Cadidate'] = int(re.findall(r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', data.loc[i,'Number Cadidate'])[0])
      # Xử lý cột Years of Experience
      data.loc[i,'Years of Experience'] = normalize_experience_job(data.loc[i,'Years of Experience'])
      # Xử lý cột Salary
      salary = min(normalize_salary_job(data.loc[i,'Salary']))
      try:
        if salary < 5000000:
          data.loc[i,'Salary'] = '1,000,000 - 5,000,000'
        elif salary < 10000000:
          data.loc[i,'Salary'] = '5,000,000 - 10,000,000'
        elif salary < 15000000:
          data.loc[i,'Salary'] = '10,000,000 - 15,000,000'
        elif salary < 20000000:
          data.loc[i,'Salary'] = '15,000,000 - 20,000,000'
        elif salary < 40000000:
          data.loc[i,'Salary'] = '20,000,000 - 40,000,000'
        else:
          data.loc[i,'Salary'] = 'Trên 40,000,000'
      except:
        data.loc[i,'Salary'] = salary
      # Xử lý cột Submission Deadline
      data.loc[i,'Submission Deadline'] = normalize_text_job(data.loc[i,'Submission Deadline'])
      # Xử lý cột Industry
      try:
        data.loc[i,'Industry'] = normalize_industry(data.loc[i,'Industry'])
      except:
        data.loc[i,'Industry'] = 'Không xác định'
    except:
      continue

  return data

In [ ]:
%%time
processing_job_data(df_job)

100%|██████████| 14634/14634 [01:00<00:00, 241.56it/s]

CPU times: user 1min, sys: 2.09 s, total: 1min 2s
Wall time: 1min


,URL Job,Job Title,Name Company,Company Overview,Company Size,Company Address,Job Description,Job Requirements,Benefits,Job Address,Job Type,Gender,Number Cadidate,Career Level,Years of Experience,Salary,Submission Deadline,Industry
0,https://www.topcv.vn/viec-lam/sale-admin-websi...,Sale Admin Website,Công ty Cổ phần Belie,DHC Việt Nam trực thuộc Công ty CP Belie – Nhà...,100-499,"HN: Tầng 9, Tòa IDMC Mỹ Đình, 18 Tôn Thất Thuy...",Thường xuyên cập nhật các thay đổi liên quan t...,"1. Học vấn, kiến thức và trình độ chuyên môn:T...",Được xét duyệt đánh giá định kỳ 2 lần/năm.Làm ...,Hà Nội,Full time,Không yêu cầu,1,Nhân viên,1-3 năm,"5,000,000 - 10,000,000",05/01/2023,Hành chính / Văn phòng
1,https://www.topcv.vn/viec-lam/thuc-tap-sinh-la...,"Thực Tập Sinh Lập Trình (No-Code, Low-Code Pla...",Công Ty Cổ Phần Global Online Branding,Thành viên thuộc Tập đoàn G-Group:Công ty Cổ p...,100-499,"Lầu 6, 235 Lý Thường Kiệt ,Phường 6 ,Quận Tân ...","Phối hợp với bộ phận lập trình No-code, Low-co...","Sinh viên năm 3, năm 4 hoặc sinh viên cao học ...",Làm việc trong môi trường văn hóa Nhật Bản chu...,Hồ Chí Minh,Full time,Không yêu cầu,2,Thực tập sinh,Không yêu cầu kinh nghiệm,"1,000,000 - 5,000,000",01/01/2023,Công nghệ thông tin
2,https://www.topcv.vn/viec-lam/hr-business-part...,HR Business Partner,Công Ty Melody Logistics,Công Ty TNHH Melody Logistics được biết đến là...,100-499,"Số 01 Nguyễn Văn Đậu Phường 5, Quận Phú Nhuận,...",Xây dựng và thực hiện chiến lược nhân sự phù h...,Tốt nghiệp đại học các chuyên ngành liên quan ...,Lương thỏa thuận theo năng lực;Làm việc trong ...,Hồ Chí Minh,Full time,Không yêu cầu,1,Trưởng/Phó phòng,5-10 năm,Thỏa thuận,28/12/2022,Nhân sự
3,https://www.topcv.vn/viec-lam/general-manager/...,General Manager,CÔNG TY TNHH DV HOLDINGS,"Khách sạn Danaciti By Dabi là khách sạn 4 sao,...",25-99,"80 Đông Kinh Nghĩa Thục, Sơn Trà, Đà Nẵng",1. Lập và triển khai kế hoạch kinh doanh- Định...,- Tốt nghiệp đại học trở lên chuyên ngành quản...,"- Tiền lương, thưởng và các khoản trợ cấp khác...",Đà Nẵng,Full time,Không yêu cầu,1,Giám đốc,3-5 năm,Thỏa thuận,31/12/2022,Khách sạn / Nhà hàng
4,https://www.topcv.vn/viec-lam/le-tan-gymasaste...,Lễ Tân Gymasaster Quận 12,CÔNG TY TNHH TM XNK NGUỒN SỐNG VIỆT,Công ty TNHH TM XNK Nguồn Sống Việt cung câ...,100-499,"235 đường 9A, khu dân cư Trung Sơn, huyện Bi...",- Trực quầy Lễ tân- Tư vấn bán hàng gói member...,- Giới tính: Ưu tiên Nữ từ 22 - 35 tuổi- Kinh ...,Thu nhập = Lương cơ bản + Thưởng KPIs + Hoa hồ...,Hồ Chí Minh,Full time,Không yêu cầu,2,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",06/01/2023,Hành chính / Văn phòng
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14629,https://timviec365.vn/viec-lam-ban-thoi-gian-t...,Việc Làm Bán Thời Gian Tại Nhà,công ty TNHH quảng cáo truyền thông xuyên việt,"Mô tả: chuyên về quảng cáo,...",Đang cập nhật,"88 Nguyễn Thái Học, Cầu Ông Lãnh, 1, Hồ Chí Minh",- Đối tượng: phù hợp với tất cả mọi người như:...,BẠN SẼ PHÙ HỢP VỚI VỊ TRÍ NÀY NẾU BẠN: - Có kh...,Các Phúc Lợi Dành Cho Bạn : -Lương : Làm việc ...,Đắk Lắk,Part time,Không yêu cầu,8,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",29/12/2028,Sinh viên làm thêm
14630,https://timviec365.vn/nhan-vien-tu-van-tin-dun...,Nhân viên tư vấn Tín dụng qua điện thoại,Cty Cổ Phần Giải Pháp Nhân Sự Áng Sáng - Light...,,Đang cập nhật,"144 Tòa Nhà Fe Credit, Cộng Hòa, Tân Bình",1.: - Sử dụng hệ thống Call-center của công ty...,- Tốt nghiệp THPT - Tuổi từ 19-33t- Kỹ năng gi...,- Lương cơ bản: 5.100.000/Tháng - Phụ cấp cơm:...,Hồ Chí Minh,Full time,Không yêu cầu,100,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",30/12/2022,nhân viên tín dụng ngân hàng
14631,https://timviec365.vn/tuyen-nhan-vien-ki-thuat...,tuyển nhân viên kỹ thuật,Công Ty Tnhh Mtv Az Company,công ty chúng tôi chuyên cung cấp các dịch vụ ...,Đang cập nhật,114 B15,công ty tuyển nhân viên bảo vệ cho tòa văn phò...,"yêu cầu ứng viên nam, tuổi từ 25 đến 55, có ki

#### **Check & Processing Missing Values**

In [ ]:
df_job.isna().sum()

URL Job                0
Job Title              0
Name Company           0
Company Overview       0
Company Size           0
Company Address        0
Job Description        0
Job Requirements       0
Benefits               0
Job Address            0
Job Type               0
Gender                 0
Number Cadidate        0
Career Level           0
Years of Experience    0
Salary                 0
Submission Deadline    0
Industry               0
dtype: int64

In [ ]:
df_job['Company Overview'].replace('', 'Đang cập nhật', inplace = True)
df_job['Job Requirements'].replace('', 'Đang cập nhật', inplace = True)
df_job['Benefits'].replace('', 'Đang cập nhật', inplace = True)
df_job['Job Address'].replace('', 'Đang cập nhật', inplace = True)

In [ ]:
df_job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14634 entries, 0 to 14633
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   URL Job              14634 non-null  object
 1   Job Title            14634 non-null  object
 2   Name Company         14634 non-null  object
 3   Company Overview     14634 non-null  object
 4   Company Size         14634 non-null  object
 5   Company Address      14634 non-null  object
 6   Job Description      14634 non-null  object
 7   Job Requirements     14634 non-null  object
 8   Benefits             14634 non-null  object
 9   Job Address          14634 non-null  object
 10  Job Type             14634 non-null  object
 11  Gender               14634 non-null  object
 12  Number Cadidate      14634 non-null  object
 13  Career Level         14634 non-null  object
 14  Years of Experience  14634 non-null  object
 15  Salary               14634 non-null  object
 16  Subm

#### **Save**

In [ ]:
# Save data
# df_job.to_csv(PATH+'JOB_DATA_TN.csv', index=False, encoding='utf-8')

## **New**

In [ ]:
def nomalize_text(data):
  #- chuyển về từ thường
  data = data.lower()
  #- Xóa dấu câu, ký tự đặc biệt
  data = re.sub('\W+',' ', data)

  return data

#### **User data**

In [ ]:
df_user = pd.read_csv(PATH+'USER_DATA_TN.csv')
df_user

,URL User,UserID,User Name,Industry,Desired Job,Workplace Desired,Desired Salary,Gender,Marriage,Age,Target,Skills,Degree,Work Experience
0,https://timviec365.vn/ung-vien/nguyen-quan-uv9...,964496,Nguyễn Quân,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,36,Với kinh nghiệm 10 năm trong lĩnh vực lái xe t...,"- Bằng lái xe hạng C, D, E và có kinh nghiệm 5...",Đơn vị đào tạo: Trường trung cấp nghề,5-10 năm
1,https://timviec365.vn/ung-vien/nguyen-chau-uv9...,986206,Nguyễn Châu,Vận tải - Lái xe,Nhân viên lái xe bằng D,Bình Dương,Thỏa thuận,Nam,Đã có gia đình,44,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trường trung cấp nghề,Trên 10 năm
2,https://timviec365.vn/ung-vien/la-thi-hoang-uv...,769632,LÃ THỊ HOÀNG,Hành chính - Văn phòng,"Nhân viên văn phòng, khu công nghiệp sử dụng t...",Bắc Giang,Thỏa thuận,Nữ,Độc thân,25,- Tìm kiếm một vị trí nhân viên hành chính và ...,- Am hiểu mọi quy trình về hành chính nhân sự ...,Mô tả công việc: CÔNG TY CỔ PHẦN DƯỢC PHẨM AN ...,1-3 năm
3,https://timviec365.vn/ung-vien/nguyen-van-bao-...,981314,Nguyễn Văn Bảo,Vận tải - Lái xe,Nhân viên lái xe hạng C,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,33,Trở thành kỹ sư tốt trong công tác sửa chữa xe...,"Có khả năng chu toàn công việc, siêng năng, tỉ...",Đơn vị đào tạo: Trung tâm dạy nghề,5-10 năm
4,https://timviec365.vn/ung-vien/nguyen-thi-tu-u...,962892,Nguyễn Thị Tứ,Quản lý đơn hàng,Nhân viên thu mua,Hồ Chí Minh,Thỏa thuận,Nữ,Độc thân,24,"- Có một công việc có nhiều cơ hội thăng tiến,...","- Trung thực, cẩn thận, có óc sáng tạo - Kỹ...",Đơn vị đào tạo: Cao Đẳng Kinh Tế Kỹ Thuật Vina...,3-5 năm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,https://timviec365.vn/ung-vien/bui-nghia-uv997...,997433,Bùi Nghĩa,Vận tải - Lái xe,Nhân viên lái xe hạng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,35,"Trở thành nhân viên lái xe cho công ty, tôi mu...","- Tiếp thu tốt các kiến thức lái xe, - Có trí ...",Đơn vị đào tạo: Trường trung cấp nghề,1-3 năm
3979,https://timviec365.vn/ung-vien/tran-thanh-son-...,964381,Trần Thanh Sơn,Vận tải - Lái xe,Nhân viên Lái xe hạng C,Khánh Hòa,Thỏa thuận,Nam,Đã có gia đình,29,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trung học,5-10 năm
3980,https://timviec365.vn/ung-vien/huynh-qui-uv969...,969486,Huỳnh Quí,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hồ Chí Minh,Thỏa thuận,Nam,Độc thân,20,Áp dụng những kiến thức về giao thông vận tải ...,"Khả năng chịu áp lực cao trong công việc, có t...",Đơn vị đào tạo: Trường trung cấp nghề,Dưới 1 năm
3981,https://timviec365.vn/ung-vien/pham-van-truong...,978744,Phạm Văn Trường,Vận tải - Lái xe,Nhân viên lái xe - Bằng Lái Xe Hạng D,Quảng Ninh,Thỏa thuận,Nam,Đã có gia đình,34,"Đáp ứng tốt các công việc của công ty, giữ gìn...","Giao tiếp tốt, thân thiện, cởi mở với mọi ngườ...",Đơn vị đào tạo: Tốt nghiệp THPT,Trên 10 năm


In [ ]:
df_user.isna().sum()

URL User             0
UserID               0
User Name            0
Industry             0
Desired Job          0
Workplace Desired    0
Desired Salary       0
Gender               0
Marriage             0
Age                  0
Target               0
Skills               0
Degree               0
Work Experience      0
dtype: int64

In [ ]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3983 entries, 0 to 3982
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   URL User           3983 non-null   object
 1   UserID             3983 non-null   int64 
 2   User Name          3983 non-null   object
 3   Industry           3983 non-null   object
 4   Desired Job        3983 non-null   object
 5   Workplace Desired  3983 non-null   object
 6   Desired Salary     3983 non-null   object
 7   Gender             3983 non-null   object
 8   Marriage           3983 non-null   object
 9   Age                3983 non-null   int64 
 10  Target             3983 non-null   object
 11  Skills             3983 non-null   object
 12  Degree             3983 non-null   object
 13  Work Experience    3983 non-null   object
dtypes: int64(2), object(12)
memory usage: 435.8+ KB


In [ ]:
# Save
# df_user.to_csv(PATH+'USER_DATA_FINAL.csv', index=False)

In [ ]:
df_user_new = df_user.copy()

for i in tqdm(range(df_user_new.shape[0])):
  # Xử lý cột Industry
  df_user_new.loc[i,'Industry'] = nomalize_text(df_user_new.loc[i,'Industry'])
  # Xử lý cột ...

df_user_new

100%|██████████| 3983/3983 [00:01<00:00, 2500.17it/s]


,URL User,UserID,User Name,Industry,Desired Job,Workplace Desired,Desired Salary,Gender,Marriage,Age,Target,Skills,Degree,Work Experience
0,https://timviec365.vn/ung-vien/nguyen-quan-uv9...,964496,Nguyễn Quân,vận tải lái xe,Nhân viên lái xe bằng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,36,Với kinh nghiệm 10 năm trong lĩnh vực lái xe t...,"- Bằng lái xe hạng C, D, E và có kinh nghiệm 5...",Đơn vị đào tạo: Trường trung cấp nghề,5-10 năm
1,https://timviec365.vn/ung-vien/nguyen-chau-uv9...,986206,Nguyễn Châu,vận tải lái xe,Nhân viên lái xe bằng D,Bình Dương,Thỏa thuận,Nam,Đã có gia đình,44,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trường trung cấp nghề,Trên 10 năm
2,https://timviec365.vn/ung-vien/la-thi-hoang-uv...,769632,LÃ THỊ HOÀNG,hành chính văn phòng,"Nhân viên văn phòng, khu công nghiệp sử dụng t...",Bắc Giang,Thỏa thuận,Nữ,Độc thân,25,- Tìm kiếm một vị trí nhân viên hành chính và ...,- Am hiểu mọi quy trình về hành chính nhân sự ...,Mô tả công việc: CÔNG TY CỔ PHẦN DƯỢC PHẨM AN ...,1-3 năm
3,https://timviec365.vn/ung-vien/nguyen-van-bao-...,981314,Nguyễn Văn Bảo,vận tải lái xe,Nhân viên lái xe hạng C,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,33,Trở thành kỹ sư tốt trong công tác sửa chữa xe...,"Có khả năng chu toàn công việc, siêng năng, tỉ...",Đơn vị đào tạo: Trung tâm dạy nghề,5-10 năm
4,https://timviec365.vn/ung-vien/nguyen-thi-tu-u...,962892,Nguyễn Thị Tứ,quản lý đơn hàng,Nhân viên thu mua,Hồ Chí Minh,Thỏa thuận,Nữ,Độc thân,24,"- Có một công việc có nhiều cơ hội thăng tiến,...","- Trung thực, cẩn thận, có óc sáng tạo - Kỹ...",Đơn vị đào tạo: Cao Đẳng Kinh Tế Kỹ Thuật Vina...,3-5 năm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,https://timviec365.vn/ung-vien/bui-nghia-uv997...,997433,Bùi Nghĩa,vận tải lái xe,Nhân viên lái xe hạng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,35,"Trở thành nhân viên lái xe cho công ty, tôi mu...","- Tiếp thu tốt các kiến thức lái xe, - Có trí ...",Đơn vị đào tạo: Trường trung cấp nghề,1-3 năm
3979,https://timviec365.vn/ung-vien/tran-thanh-son-...,964381,Trần Thanh Sơn,vận tải lái xe,Nhân viên Lái xe hạng C,Khánh Hòa,Thỏa thuận,Nam,Đã có gia đình,29,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trung học,5-10 năm
3980,https://timviec365.vn/ung-vien/huynh-qui-uv969...,969486,Huỳnh Quí,vận tải lái xe,Nhân viên lái xe bằng B2,Hồ Chí Minh,Thỏa thuận,Nam,Độc thân,20,Áp dụng những kiến thức về giao thông vận tải ...,"Khả năng chịu áp lực cao trong công việc, có t...",Đơn vị đào tạo: Trường trung cấp nghề,Dưới 1 năm
3981,https://timviec365.vn/ung-vien/pham-van-truong...,978744,Phạm Văn Trường,vận tải lái xe,Nhân viên lái xe - Bằng Lái Xe Hạng D,Quảng Ninh,Thỏa thuận,Nam,Đã có gia đình,34,"Đáp ứng tốt các công việc của công ty, giữ gìn...","Giao tiếp tốt, thân thiện, cởi mở với mọi ngườ...",Đơn vị đào tạo: Tốt nghiệp THPT,Trên 10 năm


#### **Job data**

In [ ]:
df_job = pd.read_csv(PATH+'JOB_DATA_TN.csv')
df_job

,URL Job,Job Title,Name Company,Company Overview,Company Size,Company Address,Job Description,Job Requirements,Benefits,Job Address,Job Type,Gender,Number Cadidate,Career Level,Years of Experience,Salary,Submission Deadline,Industry
0,https://www.topcv.vn/viec-lam/sale-admin-websi...,Sale Admin Website,Công ty Cổ phần Belie,DHC Việt Nam trực thuộc Công ty CP Belie – Nhà...,100-499,"HN: Tầng 9, Tòa IDMC Mỹ Đình, 18 Tôn Thất Thuy...",Thường xuyên cập nhật các thay đổi liên quan t...,"1. Học vấn, kiến thức và trình độ chuyên môn:T...",Được xét duyệt đánh giá định kỳ 2 lần/năm.Làm ...,Hà Nội,Full time,Không yêu cầu,1,Nhân viên,1-3 năm,"5,000,000 - 10,000,000",05/01/2023,Hành chính / Văn phòng
1,https://www.topcv.vn/viec-lam/thuc-tap-sinh-la...,"Thực Tập Sinh Lập Trình (No-Code, Low-Code Pla...",Công Ty Cổ Phần Global Online Branding,Thành viên thuộc Tập đoàn G-Group:Công ty Cổ p...,100-499,"Lầu 6, 235 Lý Thường Kiệt ,Phường 6 ,Quận Tân ...","Phối hợp với bộ phận lập trình No-code, Low-co...","Sinh viên năm 3, năm 4 hoặc sinh viên cao học ...",Làm việc trong môi trường văn hóa Nhật Bản chu...,Hồ Chí Minh,Full time,Không yêu cầu,2,Thực tập sinh,Không yêu cầu kinh nghiệm,"1,000,000 - 5,000,000",01/01/2023,Công nghệ thông tin
2,https://www.topcv.vn/viec-lam/hr-business-part...,HR Business Partner,Công Ty Melody Logistics,Công Ty TNHH Melody Logistics được biết đến là...,100-499,"Số 01 Nguyễn Văn Đậu Phường 5, Quận Phú Nhuận,...",Xây dựng và thực hiện chiến lược nhân sự phù h...,Tốt nghiệp đại học các chuyên ngành liên quan ...,Lương thỏa thuận theo năng lực;Làm việc trong ...,Hồ Chí Minh,Full time,Không yêu cầu,1,Trưởng/Phó phòng,5-10 năm,Thỏa thuận,28/12/2022,Nhân sự
3,https://www.topcv.vn/viec-lam/general-manager/...,General Manager,CÔNG TY TNHH DV HOLDINGS,"Khách sạn Danaciti By Dabi là khách sạn 4 sao,...",25-99,"80 Đông Kinh Nghĩa Thục, Sơn Trà, Đà Nẵng",1. Lập và triển khai kế hoạch kinh doanh- Định...,- Tốt nghiệp đại học trở lên chuyên ngành quản...,"- Tiền lương, thưởng và các khoản trợ cấp khác...",Đà Nẵng,Full time,Không yêu cầu,1,Giám đốc,3-5 năm,Thỏa thuận,31/12/2022,Khách sạn / Nhà hàng
4,https://www.topcv.vn/viec-lam/le-tan-gymasaste...,Lễ Tân Gymasaster Quận 12,CÔNG TY TNHH TM XNK NGUỒN SỐNG VIỆT,Công ty TNHH TM XNK Nguồn Sống Việt cung câ...,100-499,"235 đường 9A, khu dân cư Trung Sơn, huyện Bi...",- Trực quầy Lễ tân- Tư vấn bán hàng gói member...,- Giới tính: Ưu tiên Nữ từ 22 - 35 tuổi- Kinh ...,Thu nhập = Lương cơ bản + Thưởng KPIs + Hoa hồ...,Hồ Chí Minh,Full time,Không yêu cầu,2,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",06/01/2023,Hành chính / Văn phòng
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14629,https://timviec365.vn/viec-lam-ban-thoi-gian-t...,Việc Làm Bán Thời Gian Tại Nhà,công ty TNHH quảng cáo truyền thông xuyên việt,"Mô tả: chuyên về quảng cáo,...",Đang cập nhật,"88 Nguyễn Thái Học, Cầu Ông Lãnh, 1, Hồ Chí Minh",- Đối tượng: phù hợp với tất cả mọi người như:...,BẠN SẼ PHÙ HỢP VỚI VỊ TRÍ NÀY NẾU BẠN: - Có kh...,Các Phúc Lợi Dành Cho Bạn : -Lương : Làm việc ...,Đắk Lắk,Part time,Không yêu cầu,8,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",29/12/2028,Sinh viên làm thêm
14630,https://timviec365.vn/nhan-vien-tu-van-tin-dun...,Nhân viên tư vấn Tín dụng qua điện thoại,Cty Cổ Phần Giải Pháp Nhân Sự Áng Sáng - Light...,Đang cập nhật,Đang cập nhật,"144 Tòa Nhà Fe Credit, Cộng Hòa, Tân Bình",1.: - Sử dụng hệ thống Call-center của công ty...,- Tốt nghiệp THPT - Tuổi từ 19-33t- Kỹ năng gi...,- Lương cơ bản: 5.100.000/Tháng - Phụ cấp cơm:...,Hồ Chí Minh,Full time,Không yêu cầu,100,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",30/12/2022,nhân viên tín dụng ngân hàng
14631,https://timviec365.vn/tuyen-nhan-vien-ki-thuat...,tuyển nhân viên kỹ thuật,Công Ty Tnhh Mtv Az Company,công ty chúng tôi chuyên cung cấp các dịch vụ ...,Đang cập nhật,114 B15,công ty tuyển nhân viên bảo vệ cho tòa văn phò...,"yêu cầu ứng viên nam, tuổi từ 25 

In [ ]:
df_job.isna().sum()

URL Job                0
Job Title              0
Name Company           0
Company Overview       0
Company Size           0
Company Address        0
Job Description        0
Job Requirements       0
Benefits               0
Job Address            0
Job Type               0
Gender                 0
Number Cadidate        0
Career Level           0
Years of Experience    0
Salary                 0
Submission Deadline    0
Industry               0
dtype: int64

In [ ]:
df_job.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14634 entries, 0 to 14633
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   URL Job              14634 non-null  object
 1   Job Title            14634 non-null  object
 2   Name Company         14634 non-null  object
 3   Company Overview     14634 non-null  object
 4   Company Size         14634 non-null  object
 5   Company Address      14634 non-null  object
 6   Job Description      14634 non-null  object
 7   Job Requirements     14634 non-null  object
 8   Benefits             14634 non-null  object
 9   Job Address          14634 non-null  object
 10  Job Type             14634 non-null  object
 11  Gender               14634 non-null  object
 12  Number Cadidate      14634 non-null  int64 
 13  Career Level         14634 non-null  object
 14  Years of Experience  14634 non-null  object
 15  Salary               14634 non-null  object
 16  Subm

In [ ]:
# Save
# df_job.to_csv(PATH+'JOB_DATA_FINAL.csv', index_label='JobID')

In [ ]:
df_job_new = df_job.copy()

for i in tqdm(range(df_job_new.shape[0])):
  # Xử lý cột Industry
  df_job_new.loc[i,'Industry'] = nomalize_text(df_job_new.loc[i,'Industry'])
  # Xử lý cột ...

df_job_new

100%|██████████| 14634/14634 [00:16<00:00, 870.83it/s] 


,URL Job,Job Title,Name Company,Company Overview,Company Size,Company Address,Job Description,Job Requirements,Benefits,Job Address,Job Type,Gender,Number Cadidate,Career Level,Years of Experience,Salary,Submission Deadline,Industry
0,https://www.topcv.vn/viec-lam/sale-admin-websi...,Sale Admin Website,Công ty Cổ phần Belie,DHC Việt Nam trực thuộc Công ty CP Belie – Nhà...,100-499,"HN: Tầng 9, Tòa IDMC Mỹ Đình, 18 Tôn Thất Thuy...",Thường xuyên cập nhật các thay đổi liên quan t...,"1. Học vấn, kiến thức và trình độ chuyên môn:T...",Được xét duyệt đánh giá định kỳ 2 lần/năm.Làm ...,"- Hà Nội: 18 Tôn Thất Thuyết, Nam Từ Liêm",Full time,Không yêu cầu,1,Nhân viên,1-3 năm,"5,000,000 - 10,000,000",05/01/2023,hành chính văn phòng kinh doanh bán hàng kế to...
1,https://www.topcv.vn/viec-lam/thuc-tap-sinh-la...,"Thực Tập Sinh Lập Trình (No-Code, Low-Code Pla...",Công Ty Cổ Phần Global Online Branding,Thành viên thuộc Tập đoàn G-Group:Công ty Cổ p...,100-499,"Lầu 6, 235 Lý Thường Kiệt ,Phường 6 ,Quận Tân ...","Phối hợp với bộ phận lập trình No-code, Low-co...","Sinh viên năm 3, năm 4 hoặc sinh viên cao học ...",Làm việc trong môi trường văn hóa Nhật Bản chu...,"- Hồ Chí Minh: 235 Lý Thường Kiệt, Phường 6, Q...",Full time,Không yêu cầu,2,Thực tập sinh,Không yêu cầu kinh nghiệm,"1,000,000 - 5,000,000",01/01/2023,công nghệ thông tin it phần mềm việc làm it
2,https://www.topcv.vn/viec-lam/hr-business-part...,HR Business Partner,Công Ty Melody Logistics,Công Ty TNHH Melody Logistics được biết đến là...,100-499,"Số 01 Nguyễn Văn Đậu Phường 5, Quận Phú Nhuận,...",Xây dựng và thực hiện chiến lược nhân sự phù h...,Tốt nghiệp đại học các chuyên ngành liên quan ...,Lương thỏa thuận theo năng lực;Làm việc trong ...,- Hồ Chí Minh: Phú Nhuận,Full time,Không yêu cầu,1,Trưởng/Phó phòng,5-10 năm,Thỏa thuận,28/12/2022,nhân sự quản lý điều hành xuất nhập khẩu
3,https://www.topcv.vn/viec-lam/general-manager/...,General Manager,CÔNG TY TNHH DV HOLDINGS,"Khách sạn Danaciti By Dabi là khách sạn 4 sao,...",25-99,"80 Đông Kinh Nghĩa Thục, Sơn Trà, Đà Nẵng",1. Lập và triển khai kế hoạch kinh doanh- Định...,- Tốt nghiệp đại học trở lên chuyên ngành quản...,"- Tiền lương, thưởng và các khoản trợ cấp khác...","- Đà Nẵng: 80 Đông Kinh Nghĩa Thục, Sơn Trà",Full time,Không yêu cầu,1,Giám đốc,3-5 năm,Thỏa thuận,31/12/2022,khách sạn nhà hàng thực phẩm đồ uống spa làm đẹp
4,https://www.topcv.vn/viec-lam/le-tan-gymasaste...,Lễ Tân Gymasaster Quận 12,CÔNG TY TNHH TM XNK NGUỒN SỐNG VIỆT,Công ty TNHH TM XNK Nguồn Sống Việt cung câ...,100-499,"235 đường 9A, khu dân cư Trung Sơn, huyện Bi...",- Trực quầy Lễ tân- Tư vấn bán hàng gói member...,- Giới tính: Ưu tiên Nữ từ 22 - 35 tuổi- Kinh ...,Thu nhập = Lương cơ bản + Thưởng KPIs + Hoa hồ...,"- Hồ Chí Minh: Gymaster 99 Phan Văn Hớn, Quận 12",Full time,Không yêu cầu,2,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",06/01/2023,hành chính văn phòng dịch vụ khách hàng tư vấn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14629,https://timviec365.vn/viec-lam-ban-thoi-gian-t...,Việc Làm Bán Thời Gian Tại Nhà,công ty TNHH quảng cáo truyền thông xuyên việt,"Mô tả: chuyên về quảng cáo,...",Đang cập nhật,"88 Nguyễn Thái Học, Cầu Ông Lãnh, 1, Hồ Chí Minh",- Đối tượng: phù hợp với tất cả mọi người như:...,BẠN SẼ PHÙ HỢP VỚI VỊ TRÍ NÀY NẾU BẠN: - Có kh...,Các Phúc Lợi Dành Cho Bạn : -Lương : Làm việc ...,Đắk Lắk,Part time,Không yêu cầu,8,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",29/12/2028,sinh viên làm thêm
14630,https://timviec365.vn/nhan-vien-tu-van-tin-dun...,Nhân viên tư vấn Tín dụng qua điện thoại,Cty Cổ Phần Giải Pháp Nhân Sự Áng Sáng - Light...,Đang cập nhật,Đang cập nhật,"144 Tòa Nhà Fe Credit, Cộng Hòa, Tân Bình",1.: - Sử dụng hệ thống Call-center của công ty...,- Tốt nghiệp THPT - Tuổi từ 19-33t- Kỹ năng gi...,- Lương cơ bản: 5.100.000/Tháng - Phụ cấp cơm:...,Hồ Chí Minh,Full time,Không yêu cầu,100,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",3

## **EDA**

#### **User data**

In [3]:
df_user = pd.read_csv(PATH+'USER_DATA_FINAL.csv')
df_user

,URL User,UserID,User Name,Industry,Desired Job,Workplace Desired,Desired Salary,Gender,Marriage,Age,Target,Skills,Degree,Work Experience
0,https://timviec365.vn/ung-vien/nguyen-quan-uv9...,964496,Nguyễn Quân,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,36,Với kinh nghiệm 10 năm trong lĩnh vực lái xe t...,"- Bằng lái xe hạng C, D, E và có kinh nghiệm 5...",Đơn vị đào tạo: Trường trung cấp nghề,5-10 năm
1,https://timviec365.vn/ung-vien/nguyen-chau-uv9...,986206,Nguyễn Châu,Vận tải - Lái xe,Nhân viên lái xe bằng D,Bình Dương,Thỏa thuận,Nam,Đã có gia đình,44,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trường trung cấp nghề,Trên 10 năm
2,https://timviec365.vn/ung-vien/la-thi-hoang-uv...,769632,LÃ THỊ HOÀNG,Hành chính - Văn phòng,"Nhân viên văn phòng, khu công nghiệp sử dụng t...",Bắc Giang,Thỏa thuận,Nữ,Độc thân,25,- Tìm kiếm một vị trí nhân viên hành chính và ...,- Am hiểu mọi quy trình về hành chính nhân sự ...,Mô tả công việc: CÔNG TY CỔ PHẦN DƯỢC PHẨM AN ...,1-3 năm
3,https://timviec365.vn/ung-vien/nguyen-van-bao-...,981314,Nguyễn Văn Bảo,Vận tải - Lái xe,Nhân viên lái xe hạng C,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,33,Trở thành kỹ sư tốt trong công tác sửa chữa xe...,"Có khả năng chu toàn công việc, siêng năng, tỉ...",Đơn vị đào tạo: Trung tâm dạy nghề,5-10 năm
4,https://timviec365.vn/ung-vien/nguyen-thi-tu-u...,962892,Nguyễn Thị Tứ,Quản lý đơn hàng,Nhân viên thu mua,Hồ Chí Minh,Thỏa thuận,Nữ,Độc thân,24,"- Có một công việc có nhiều cơ hội thăng tiến,...","- Trung thực, cẩn thận, có óc sáng tạo - Kỹ...",Đơn vị đào tạo: Cao Đẳng Kinh Tế Kỹ Thuật Vina...,3-5 năm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,https://timviec365.vn/ung-vien/bui-nghia-uv997...,997433,Bùi Nghĩa,Vận tải - Lái xe,Nhân viên lái xe hạng B2,Hà Nội,Thỏa thuận,Nam,Đã có gia đình,35,"Trở thành nhân viên lái xe cho công ty, tôi mu...","- Tiếp thu tốt các kiến thức lái xe, - Có trí ...",Đơn vị đào tạo: Trường trung cấp nghề,1-3 năm
3979,https://timviec365.vn/ung-vien/tran-thanh-son-...,964381,Trần Thanh Sơn,Vận tải - Lái xe,Nhân viên Lái xe hạng C,Khánh Hòa,Thỏa thuận,Nam,Đã có gia đình,29,Gắn bó lâu dài với công việc cũng như doanh ng...,Có chí tiến thủ đối với công việc Có khả năng ...,Đơn vị đào tạo: Trung học,5-10 năm
3980,https://timviec365.vn/ung-vien/huynh-qui-uv969...,969486,Huỳnh Quí,Vận tải - Lái xe,Nhân viên lái xe bằng B2,Hồ Chí Minh,Thỏa thuận,Nam,Độc thân,20,Áp dụng những kiến thức về giao thông vận tải ...,"Khả năng chịu áp lực cao trong công việc, có t...",Đơn vị đào tạo: Trường trung cấp nghề,Dưới 1 năm
3981,https://timviec365.vn/ung-vien/pham-van-truong...,978744,Phạm Văn Trường,Vận tải - Lái xe,Nhân viên lái xe - Bằng Lái Xe Hạng D,Quảng Ninh,Thỏa thuận,Nam,Đã có gia đình,34,"Đáp ứng tốt các công việc của công ty, giữ gìn...","Giao tiếp tốt, thân thiện, cởi mở với mọi ngườ...",Đơn vị đào tạo: Tốt nghiệp THPT,Trên 10 năm


#### **Job data**

In [ ]:
df_job = pd.read_csv(PATH+'JOB_DATA_FINAL.csv')
df_job

,JobID,URL Job,Job Title,Name Company,Company Overview,Company Size,Company Address,Job Description,Job Requirements,Benefits,Job Address,Job Type,Gender,Number Cadidate,Career Level,Years of Experience,Salary,Submission Deadline,Industry
0,0,https://www.topcv.vn/viec-lam/sale-admin-websi...,Sale Admin Website,Công ty Cổ phần Belie,DHC Việt Nam trực thuộc Công ty CP Belie – Nhà...,100-499,"HN: Tầng 9, Tòa IDMC Mỹ Đình, 18 Tôn Thất Thuy...",Thường xuyên cập nhật các thay đổi liên quan t...,"1. Học vấn, kiến thức và trình độ chuyên môn:T...",Được xét duyệt đánh giá định kỳ 2 lần/năm.Làm ...,Hà Nội,Full time,Không yêu cầu,1,Nhân viên,1-3 năm,"5,000,000 - 10,000,000",05/01/2023,Hành chính / Văn phòng
1,1,https://www.topcv.vn/viec-lam/thuc-tap-sinh-la...,"Thực Tập Sinh Lập Trình (No-Code, Low-Code Pla...",Công Ty Cổ Phần Global Online Branding,Thành viên thuộc Tập đoàn G-Group:Công ty Cổ p...,100-499,"Lầu 6, 235 Lý Thường Kiệt ,Phường 6 ,Quận Tân ...","Phối hợp với bộ phận lập trình No-code, Low-co...","Sinh viên năm 3, năm 4 hoặc sinh viên cao học ...",Làm việc trong môi trường văn hóa Nhật Bản chu...,Hồ Chí Minh,Full time,Không yêu cầu,2,Thực tập sinh,Không yêu cầu kinh nghiệm,"1,000,000 - 5,000,000",01/01/2023,Công nghệ thông tin
2,2,https://www.topcv.vn/viec-lam/hr-business-part...,HR Business Partner,Công Ty Melody Logistics,Công Ty TNHH Melody Logistics được biết đến là...,100-499,"Số 01 Nguyễn Văn Đậu Phường 5, Quận Phú Nhuận,...",Xây dựng và thực hiện chiến lược nhân sự phù h...,Tốt nghiệp đại học các chuyên ngành liên quan ...,Lương thỏa thuận theo năng lực;Làm việc trong ...,Hồ Chí Minh,Full time,Không yêu cầu,1,Trưởng/Phó phòng,5-10 năm,Thỏa thuận,28/12/2022,Nhân sự
3,3,https://www.topcv.vn/viec-lam/general-manager/...,General Manager,CÔNG TY TNHH DV HOLDINGS,"Khách sạn Danaciti By Dabi là khách sạn 4 sao,...",25-99,"80 Đông Kinh Nghĩa Thục, Sơn Trà, Đà Nẵng",1. Lập và triển khai kế hoạch kinh doanh- Định...,- Tốt nghiệp đại học trở lên chuyên ngành quản...,"- Tiền lương, thưởng và các khoản trợ cấp khác...",Đà Nẵng,Full time,Không yêu cầu,1,Giám đốc,3-5 năm,Thỏa thuận,31/12/2022,Khách sạn / Nhà hàng
4,4,https://www.topcv.vn/viec-lam/le-tan-gymasaste...,Lễ Tân Gymasaster Quận 12,CÔNG TY TNHH TM XNK NGUỒN SỐNG VIỆT,Công ty TNHH TM XNK Nguồn Sống Việt cung câ...,100-499,"235 đường 9A, khu dân cư Trung Sơn, huyện Bi...",- Trực quầy Lễ tân- Tư vấn bán hàng gói member...,- Giới tính: Ưu tiên Nữ từ 22 - 35 tuổi- Kinh ...,Thu nhập = Lương cơ bản + Thưởng KPIs + Hoa hồ...,Hồ Chí Minh,Full time,Không yêu cầu,2,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",06/01/2023,Hành chính / Văn phòng
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14629,14629,https://timviec365.vn/viec-lam-ban-thoi-gian-t...,Việc Làm Bán Thời Gian Tại Nhà,công ty TNHH quảng cáo truyền thông xuyên việt,"Mô tả: chuyên về quảng cáo,...",Đang cập nhật,"88 Nguyễn Thái Học, Cầu Ông Lãnh, 1, Hồ Chí Minh",- Đối tượng: phù hợp với tất cả mọi người như:...,BẠN SẼ PHÙ HỢP VỚI VỊ TRÍ NÀY NẾU BẠN: - Có kh...,Các Phúc Lợi Dành Cho Bạn : -Lương : Làm việc ...,Đắk Lắk,Part time,Không yêu cầu,8,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",29/12/2028,Sinh viên làm thêm
14630,14630,https://timviec365.vn/nhan-vien-tu-van-tin-dun...,Nhân viên tư vấn Tín dụng qua điện thoại,Cty Cổ Phần Giải Pháp Nhân Sự Áng Sáng - Light...,Đang cập nhật,Đang cập nhật,"144 Tòa Nhà Fe Credit, Cộng Hòa, Tân Bình",1.: - Sử dụng hệ thống Call-center của công ty...,- Tốt nghiệp THPT - Tuổi từ 19-33t- Kỹ năng gi...,- Lương cơ bản: 5.100.000/Tháng - Phụ cấp cơm:...,Hồ Chí Minh,Full time,Không yêu cầu,100,Nhân viên,Không yêu cầu kinh nghiệm,"5,000,000 - 10,000,000",30/12/2022,nhân viên tín dụng ngân hàng
14631,14631,https://timviec365.vn/tuyen-nhan-vien-ki-thuat...,tuyển nhân viên kỹ thuật,Công Ty Tnhh Mtv Az Company,công ty chúng tôi chuyên cung cấp các dịch vụ ...,Đang cập nhật,114 B15,công ty tuyển nhân viên bảo vệ cho tòa văn phò